In [34]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Oana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
import torch
import torch.nn as nn
import numpy as np
import time

class InferSent(nn.Module):

    def __init__(self, config):
        super(InferSent, self).__init__()
        self.bsize = config['bsize']
        self.word_emb_dim = config['word_emb_dim']
        self.enc_lstm_dim = config['enc_lstm_dim']
        self.pool_type = config['pool_type']
        self.dpout_model = config['dpout_model']
        self.version = 1 if 'version' not in config else config['version']

        self.enc_lstm = nn.LSTM(
            input_size=self.word_emb_dim,
            hidden_size=self.enc_lstm_dim,
            num_layers=1,
            bidirectional=True,
            dropout=self.dpout_model
        )

        assert self.version in [1, 2]
        if self.version == 1:
            self.bos = '<s>'
            self.eos = '</s>'
            self.max_pad = True
            self.moses_tok = False
        elif self.version == 2:
            self.bos = '<p>'
            self.eos = '</p>'
            self.max_pad = False
            self.moses_tok = True

        # Strat de regresie pentru similaritate
        self.regressor = nn.Linear(2 * self.enc_lstm_dim * 2, 1)  # 2*lstm_dim * 2 pentru concatenarea premisei și hypothesis

    def is_cuda(self):
        # Verifică dacă modelul este pe GPU
        return next(self.parameters()).is_cuda

    def forward(self, sent_tuple):
        sent, sent_len = sent_tuple  # sent: (batch_size, seq_len, emb_dim); sent_len: (batch_size)

        # Sort by length in descending order
        sent_len_sorted, idx_sort = torch.sort(sent_len, descending=True)
        sent_sorted = sent.index_select(0, idx_sort)

        # Pack the sequence
        sent_packed = nn.utils.rnn.pack_padded_sequence(sent_sorted, sent_len_sorted.cpu(), batch_first=True, enforce_sorted=True)
        sent_output_packed, _ = self.enc_lstm(sent_packed)  # sent_output_packed: PackedSequence; _ : hidden state

        # Unpack the sequence
        sent_output, _ = nn.utils.rnn.pad_packed_sequence(sent_output_packed, batch_first=True)  # sent_output: (batch_size, seq_len, 2*enc_lstm_dim)

        # Unsort to original order
        _, idx_unsort = torch.sort(idx_sort, descending=False)
        sent_output = sent_output.index_select(0, idx_unsort)
        sent_len_sorted = sent_len_sorted.index_select(0, idx_unsort)

        # Pooling
        if self.pool_type == "mean":
            # Calculate the sum over the sequence dimension and divide by the lengths
            sent_len_unsorted = sent_len.index_select(0, idx_unsort).float().unsqueeze(1)  # (batch_size, 1)
            emb = torch.sum(sent_output, dim=1) / sent_len_unsorted  # (batch_size, 2*enc_lstm_dim)
        elif self.pool_type == "max":
            if not self.max_pad:
                sent_output[sent_output == 0] = -1e9
            emb, _ = torch.max(sent_output, dim=1)  # (batch_size, 2*enc_lstm_dim)

        return emb

    def encode_sentence_pair(self, premise_tuple, hypothesis_tuple):
        encoded_premise = self.forward(premise_tuple)
        encoded_hypothesis = self.forward(hypothesis_tuple)
        combined = torch.cat((encoded_premise, encoded_hypothesis), dim=1)  # (batch_size, 4*enc_lstm_dim)
        return combined

    def regress_similarity(self, combined):
        similarity = self.regressor(combined)
        return similarity.squeeze(1)  # Returnează un tensor de dimensiune (batch_size)

    def set_w2v_path(self, w2v_path):
        self.w2v_path = w2v_path

    def get_word_dict(self, sentences, tokenize=True):
        # Crează vocabularul de cuvinte
        word_dict = {}
        sentences = [s.split() if not tokenize else self.tokenize(s) for s in sentences]
        for sent in sentences:
            for word in sent:
                if word not in word_dict:
                    word_dict[word] = ''
        word_dict[self.bos] = ''
        word_dict[self.eos] = ''
        return word_dict

    def get_w2v(self, word_dict):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # Crează word_vec cu vectori w2v
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                split_line = line.strip().split(' ')
                word = split_line[0]
                vec = np.array([float(val) for val in split_line[1:]], dtype='float32')
                if word in word_dict:
                    word_vec[word] = vec
        print('Found %s/%s words with w2v vectors' % (len(word_vec), len(word_dict)))
        return word_vec

    def get_w2v_k(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # Crează word_vec cu primii K vectori w2v
        k = 0
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                if k > K and all([w in word_vec for w in [self.bos, self.eos]]):
                    break
                split_line = line.strip().split(' ')
                word = split_line[0]
                vec = np.array([float(val) for val in split_line[1:]], dtype='float32')
                if k <= K or word in [self.bos, self.eos]:
                    word_vec[word] = vec
                    k += 1
        return word_vec

    def build_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        word_dict = self.get_word_dict(sentences, tokenize)
        self.word_vec = self.get_w2v(word_dict)
        print('Vocab size : %s' % (len(self.word_vec)))

    def build_vocab_k_words(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        self.word_vec = self.get_w2v_k(K)
        print('Vocab size : %s' % (len(self.word_vec)))

    def update_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'warning : w2v path not set'
        assert hasattr(self, 'word_vec'), 'build_vocab before updating it'
        word_dict = self.get_word_dict(sentences, tokenize)

        # Păstrează doar cuvintele noi
        new_words = {word: '' for word in word_dict if word not in self.word_vec}
        if new_words:
            new_word_vec = self.get_w2v(new_words)
            self.word_vec.update(new_word_vec)
            print('New vocab size : %s (added %s words)'% (len(self.word_vec), len(new_word_vec)))
        else:
            print('No new words to add.')

    
    def get_batch(self, batch):
        # Determină dimensiunile batch-ului
        batch_size = len(batch)
        max_len = max(len(s) for s in batch)
        
        # Inițializează un array de numpy pentru embed
        embed = np.zeros((batch_size, max_len, self.word_emb_dim), dtype=np.float32)
        
        for i, sentence in enumerate(batch):
            for j, word in enumerate(sentence):
                embed[i, j, :] = self.word_vec.get(word, self.word_vec.get('<unk>', np.zeros(self.word_emb_dim)))
        
        return torch.from_numpy(embed)  # Returnează un tensor PyTorch

    def tokenize(self, s):
        # Folosește spaCy pentru tokenizare
        return preprocess_sentence_spacy(s)

    def prepare_samples(self, sentences, bsize, tokenize, verbose):
        # Adaugă <s> și </s> și tokenizează dacă este necesar
        sentences = [
            [self.bos] + s + [self.eos] for s in sentences
        ]
        n_w = np.sum([len(x) for x in sentences])

        # Filtrează cuvintele fără vectori w2v
        for i in range(len(sentences)):
            s_f = [word for word in sentences[i] if word in self.word_vec]
            if not s_f:
                import warnings
                warnings.warn('No words in "%s" (idx=%s) have w2v vectors. Replacing by "</s>"..' % (sentences[i], i))
                s_f = [self.eos]
            sentences[i] = s_f

        lengths = np.array([len(s) for s in sentences])
        n_wk = np.sum(lengths)
        if verbose:
            print('Nb words kept : %s/%s (%.1f%%)' % (n_wk, n_w, 100.0 * n_wk / n_w))

        # Sortează propozițiile după lungime descrescătoare
        lengths_sorted, idx_sort = torch.sort(torch.tensor(lengths), descending=True)
        sentences_sorted = [sentences[i] for i in idx_sort]

        if verbose:
            print(f"First 5 sorted sentence lengths: {lengths_sorted[:5].tolist()}")
            print(f"First 5 sorted sentences: {sentences_sorted[:5]}")

        return sentences_sorted, lengths_sorted.numpy(), idx_sort.numpy()

    def encode(self, sentences, bsize=64, tokenize=True, verbose=False):
        tic = time.time()
        sentences_sorted, lengths_sorted, idx_sort = self.prepare_samples(
                        sentences, bsize, tokenize, verbose)

        embeddings = []
        for stidx in range(0, len(sentences_sorted), bsize):
            batch_sentences = sentences_sorted[stidx:stidx + bsize]
            batch = self.get_batch(batch_sentences)
            if self.is_cuda():
                batch = batch.cuda()
            lengths = lengths_sorted[stidx:stidx + bsize]
            with torch.no_grad():
                batch_output = self.forward((batch, torch.tensor(lengths).cuda() if self.is_cuda() else torch.tensor(lengths)))
                embeddings.append(batch_output.cpu().numpy())
        embeddings = np.vstack(embeddings)

        # Unsort
        idx_unsort = np.argsort(idx_sort)
        embeddings = embeddings[idx_unsort]

        if verbose:
            print('Speed : %.1f sentences/s (%s mode, bsize=%s)' % (
                    len(embeddings)/(time.time()-tic),
                    'gpu' if self.is_cuda() else 'cpu', bsize))
        return embeddings

    def visualize(self, sent, tokenize=True):
        sent = sent.split() if not tokenize else self.tokenize(sent)
        sent = [self.bos] + [word for word in sent if word in self.word_vec] + [self.eos]

        if len(sent) == 2 and sent[0] == self.bos and sent[1] == self.eos:
            import warnings
            warnings.warn('No words in "%s" have w2v vectors. Replacing by "%s %s"..' % (sent, self.bos, self.eos))
        
        batch = self.get_batch([sent])
        
        if self.is_cuda():
            batch = batch.cuda()
        output = self.forward((batch, torch.tensor([len(sent)]).cuda() if self.is_cuda() else torch.tensor([len(sent)])))
        output, idxs = torch.max(output, dim=1)
        
        idxs = idxs.cpu().numpy()
        argmaxs = [np.sum((idxs == k)) for k in range(len(sent))]

        # visualize model
        import matplotlib.pyplot as plt
        x = range(len(sent))
        y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
        plt.xticks(x, sent, rotation=45)
        plt.bar(x, y)
        plt.ylabel('%')
        plt.title('Visualisation of words importance')
        plt.show()

        return output, idxs


In [ ]:
import pandas as pd
import spacy
import os

# Încarcă modelul de limbă în spaCy
nlp = spacy.load('en_core_web_sm')

def preprocess_sentence_spacy(sentence, tokenize=True):
    if tokenize:
        doc = nlp(sentence.lower())
        return [token.text for token in doc]
    else:
        return sentence.lower().split()

# Încarcă setul de date
data = pd.read_csv(r'C:\facultate an 3\projects-simquery\data\sts_train.csv', delimiter='\t')

# Filtrează datele pentru a elimina rândurile cu valori lipsă
data = data.dropna(subset=['sent_1', 'sent_2', 'sim'])

# Extrage propozițiile și scorurile
sentences_1 = data['sent_1'].tolist()
sentences_2 = data['sent_2'].tolist()
similarities = data['sim'].tolist()

# Tokenizează propozițiile folosind spaCy
sentences_1 = [preprocess_sentence_spacy(s) for s in sentences_1]
sentences_2 = [preprocess_sentence_spacy(s) for s in sentences_2]

print("Tokenization successful using spaCy!")


Tokenization successful using spaCy!


In [37]:
import torch
from torch.utils.data import Dataset

class SimilarityDataset(Dataset):
    def __init__(self, sentences1, sentences2, similarities, word_vec):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.similarities = similarities
        self.word_vec = word_vec

    def __len__(self):
        return len(self.similarities)

    def __getitem__(self, idx):
        premise = self.sentences1[idx]
        hypothesis = self.sentences2[idx]
        similarity = self.similarities[idx]
        
        # Convertim propozițiile în vectori
        premise_np = np.array([self.word_vec.get(word, self.word_vec.get('<unk>', np.zeros(300))) for word in premise], dtype='float32')
        hypothesis_np = np.array([self.word_vec.get(word, self.word_vec.get('<unk>', np.zeros(300))) for word in hypothesis], dtype='float32')
        
        premise_tensor = torch.from_numpy(premise_np).float()
        hypothesis_tensor = torch.from_numpy(hypothesis_np).float()
        similarity_tensor = torch.tensor(similarity, dtype=torch.float)
        
        return premise_tensor, hypothesis_tensor, similarity_tensor


In [38]:
import torch.nn as nn

def collate_fn(batch):
    premises, hypotheses, similarities = zip(*batch)
    
    # Pad propozițiile
    premises_padded = nn.utils.rnn.pad_sequence(premises, batch_first=True)
    hypotheses_padded = nn.utils.rnn.pad_sequence(hypotheses, batch_first=True)
    
    # Calculează lungimile
    lengths_premises = torch.tensor([len(p) for p in premises], dtype=torch.long)
    lengths_hypotheses = torch.tensor([len(h) for h in hypotheses], dtype=torch.long)
    
    similarities = torch.tensor(similarities, dtype=torch.float)
    
    return premises_padded, lengths_premises, hypotheses_padded, lengths_hypotheses, similarities


In [39]:
import numpy as np

# Configurarea modelului
model_version = 1  # 1 pentru GloVe, 2 pentru fastText
W2V_PATH = r"C:\facultate an 3\projects-simquery\GloVe\glove.840B.300d.txt" if model_version == 1 else r"C:\facultate an 3\projects-simquery\fastText\crawl-300d-2M.vec"

params_model = {
    'bsize': 64,
    'word_emb_dim': 300,
    'enc_lstm_dim': 256,
    'pool_type': 'mean',
    'dpout_model': 0.3,
    'version': model_version
}

# Inițializarea modelului
model = InferSent(params_model)
model.set_w2v_path(W2V_PATH)

# Construiește vocabularul cu K cele mai frecvente cuvinte
K = 100000
model.build_vocab_k_words(K=K)
print(f"Vocab size : {len(model.word_vec)}")

# Adaugă token-ul '<unk>' dacă nu este deja în vocabular
if '<unk>' not in model.word_vec:
    model.word_vec['<unk>'] = np.zeros(params_model['word_emb_dim'], dtype='float32')  # Sau poți folosi un vector aleator

# Mută modelul pe GPU (dacă este disponibil)
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
    print("Modelul este pe GPU.")
else:
    print("Modelul este pe CPU.")


c:\facultate an 3\projects-simquery\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Vocab size : 100002
Vocab size : 100002
Modelul este pe CPU.


In [40]:
from sklearn.model_selection import train_test_split

# Împărțirea datelor în seturi de antrenament și validare
train_s1, val_s1, train_s2, val_s2, train_sim, val_sim = train_test_split(
    sentences_1, sentences_2, similarities, test_size=0.1, random_state=42)


In [41]:
from torch.utils.data import DataLoader

# Crearea dataset-urilor
train_dataset = SimilarityDataset(train_s1, train_s2, train_sim, model.word_vec)
val_dataset = SimilarityDataset(val_s1, val_s2, val_sim, model.word_vec)

# Crearea DataLoader-elor
train_loader = DataLoader(train_dataset, batch_size=params_model['bsize'], shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=params_model['bsize'], shuffle=False, collate_fn=collate_fn)


In [42]:
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)




In [43]:
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
import numpy as np
import time
import pandas as pd
import spacy
import os
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import wandb
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt

# Definiția clasei InferSent și alte funcții aici...

def calculate_regression_metrics(preds, targets):
    """
    Calculează metricile standard de regresie.
    
    Args:
        preds (list or np.array): Predicțiile modelului.
        targets (list or np.array): Valorile reale.
    
    Returns:
        tuple: MAE, RMSE, R2, Pearson, Spearman
    """
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    r2 = r2_score(targets, preds)
    pearson = pearsonr(targets, preds)[0]
    spearman = spearmanr(targets, preds)[0]
    return mae, rmse, r2, pearson, spearman

def calculate_accuracy(preds, targets, threshold=0.5):
    """
    Calculează acuratețea ca procentajul predicțiilor care sunt în interiorul
    unui interval definit de threshold față de valorile reale.
    
    Args:
        preds (list or np.array): Predicțiile modelului.
        targets (list or np.array): Valorile reale.
        threshold (float): Pragul de toleranță.
    
    Returns:
        float: Acuratețea în procente.
    """
    preds = np.array(preds)
    targets = np.array(targets)
    correct = np.abs(preds - targets) <= threshold
    accuracy = np.mean(correct) * 100
    return accuracy

# Restul codului de preprocesare și inițializare a modelului...

# Inițializarea wandb
wandb.init(
    project="inferSent-project",  # Înlocuiește cu numele proiectului tău pe wandb
    config={
        "learning_rate": 0.001,
        "epochs": 10,
        "batch_size": 64,
        "model_version": model_version,
        "word_emb_dim": params_model['word_emb_dim'],
        "enc_lstm_dim": params_model['enc_lstm_dim'],
        "pool_type": params_model['pool_type'],
        "dropout": params_model['dpout_model'],
        "accuracy_threshold": 0.5  # Pragul pentru acuratețe
    }
)

config = wandb.config

num_epochs = config.epochs
threshold = config.accuracy_threshold
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Antrenament
    model.train()
    total_loss = 0.0
    train_preds = []
    train_targets = []
    
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        premises_padded, lengths_premises, hypotheses_padded, lengths_hypotheses, similarities = batch
        if use_cuda:
            premises_padded = premises_padded.cuda()
            lengths_premises = lengths_premises.cuda()
            hypotheses_padded = hypotheses_padded.cuda()
            lengths_hypotheses = lengths_hypotheses.cuda()
            similarities = similarities.cuda()
        
        optimizer.zero_grad()
        
        # Encodează perechea de propoziții
        combined = model.encode_sentence_pair(
            (premises_padded, lengths_premises),
            (hypotheses_padded, lengths_hypotheses)
        )
        
        # Prezice similaritatea
        preds = model.regress_similarity(combined)
        
        # Calculează pierderea
        loss = criterion(preds, similarities)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Colectează predicțiile și țintele pentru metrici suplimentare
        train_preds.extend(preds.detach().cpu().numpy())
        train_targets.extend(similarities.detach().cpu().numpy())
    
    avg_train_loss = total_loss / len(train_loader)
    
    # Calcularea metricilor pentru antrenament
    train_mae, train_rmse, train_r2, train_pearson, train_spearman = calculate_regression_metrics(train_preds, train_targets)
    train_accuracy = calculate_accuracy(train_preds, train_targets, threshold=threshold)
    
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg Training Loss: {avg_train_loss:.4f} - "
          f"MAE: {train_mae:.4f} - RMSE: {train_rmse:.4f} - "
          f"R2: {train_r2:.4f} - Pearson: {train_pearson:.4f} - "
          f"Spearman: {train_spearman:.4f} - Accuracy: {train_accuracy:.2f}%")
    
    # Logare metrici în wandb pentru antrenament
    wandb.log({
        "Epoch": epoch + 1,
        "Training Loss": avg_train_loss,
        "Training MAE": train_mae,
        "Training RMSE": train_rmse,
        "Training R2": train_r2,
        "Training Pearson": train_pearson,
        "Training Spearman": train_spearman,
        "Training Accuracy": train_accuracy
    })
    
    # Validare
    model.eval()
    total_val_loss = 0.0
    val_preds = []
    val_targets = []
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            premises_padded, lengths_premises, hypotheses_padded, lengths_hypotheses, similarities = batch
            if use_cuda:
                premises_padded = premises_padded.cuda()
                lengths_premises = lengths_premises.cuda()
                hypotheses_padded = hypotheses_padded.cuda()
                lengths_hypotheses = lengths_hypotheses.cuda()
                similarities = similarities.cuda()
            
            combined = model.encode_sentence_pair(
                (premises_padded, lengths_premises),
                (hypotheses_padded, lengths_hypotheses)
            )
            preds = model.regress_similarity(combined)
            loss = criterion(preds, similarities)
            total_val_loss += loss.item()
            
            # Colectează predicțiile și țintele pentru metrici suplimentare
            val_preds.extend(preds.detach().cpu().numpy())
            val_targets.extend(similarities.detach().cpu().numpy())
    
    avg_val_loss = total_val_loss / len(val_loader)
    
    # Calcularea metricilor pentru validare
    val_mae, val_rmse, val_r2, val_pearson, val_spearman = calculate_regression_metrics(val_preds, val_targets)
    val_accuracy = calculate_accuracy(val_preds, val_targets, threshold=threshold)
    
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg Validation Loss: {avg_val_loss:.4f} - "
          f"MAE: {val_mae:.4f} - RMSE: {val_rmse:.4f} - "
          f"R2: {val_r2:.4f} - Pearson: {val_pearson:.4f} - "
          f"Spearman: {val_spearman:.4f} - Accuracy: {val_accuracy:.2f}%")
    
    # Logare metrici în wandb pentru validare
    wandb.log({
        "Epoch": epoch + 1,
        "Validation Loss": avg_val_loss,
        "Validation MAE": val_mae,
        "Validation RMSE": val_rmse,
        "Validation R2": val_r2,
        "Validation Pearson": val_pearson,
        "Validation Spearman": val_spearman,
        "Validation Accuracy": val_accuracy
    })
    
    # Salvează modelul dacă pierderea pe validare este mai mică
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        model_path = f"infersent{model_version}_best.pth"
        torch.save(model.state_dict(), model_path)
        print("Modelul a fost salvat pentru performanță mai bună pe validare.")
        
        # Salvează modelul în wandb folosind Artifacts
        artifact = wandb.Artifact('best_model', type='model')
        artifact.add_file(model_path)
        wandb.log_artifact(artifact)

# Închide wandb la finalul antrenamentului
wandb.finish()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Oana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training Epoch 1/10: 100%|██████████| 81/81 [00:18<00:00,  4.33it/s]


Epoch 1/10 - Avg Training Loss: 1.9835 - MAE: 1.9844 - RMSE: 2.3824 - R2: -1.6472 - Pearson: 0.1136 - Spearman: 0.0731 - Accuracy: 16.64%


Validation: 100%|██████████| 9/9 [00:00<00:00, 23.44it/s]


Epoch 1/10 - Avg Validation Loss: 1.6133 - MAE: 1.6132 - RMSE: 1.9983 - R2: -0.8489 - Pearson: 0.1478 - Spearman: 0.1197 - Accuracy: 19.48%
Modelul a fost salvat pentru performanță mai bună pe validare.


...

Training Epoch 10/10: 100%|██████████| 81/81 [00:18<00:00,  4.32it/s]


Epoch 10/10 - Avg Training Loss: 1.2991 - MAE: 1.2998 - RMSE: 1.5830 - R2: -0.1688 - Pearson: 0.2096 - Spearman: 0.2012 - Accuracy: 22.62%


Validation: 100%|██████████| 9/9 [00:00<00:00, 25.64it/s]


Epoch 10/10 - Avg Validation Loss: 1.3129 - MAE: 1.3127 - RMSE: 1.5803 - R2: -0.1564 - Pearson: 0.2031 - Spearman: 0.1803 - Accuracy: 19.48%
Modelul a fost salvat pentru performanță mai bună pe validare.


Epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
Training Accuracy,▁▅▅▆▆▇█▅▇█
Training Loss,█▄▂▂▂▁▁▁▁▁
Training MAE,█▄▂▂▂▁▁▁▁▁
Training Pearson,▁▅▁▂▂▆▅▆██
Training R2,▁▅▇▇██████
Training RMSE,█▄▂▂▂▁▁▁▁▁
Training Spearman,▁▄▂▃▃▆▆▆██
Validation Accuracy,▁█▄▃▂▃▂▇▁▁
Validation Loss,█▅▃▃▂▂▁▁▁▁
Validation MAE,█▅▃▃▂▂▁▁▁▁


In [44]:
import wandb
import torch
import os

# Reinițializează wandb
wandb.init(project="inferSent-project")

# Folosește Artifact-ul pentru a descărca modelul
artifact = wandb.use_artifact('best_model:model')  # 'model' este tipul Artifact-ului specificat anterior
artifact_dir = artifact.download()

# Încarcă starea modelului
model_path = os.path.join(artifact_dir, 'infersent1_best.pth')  # Asigură-te că numele fișierului este corect
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))  # Schimbă 'cpu' la 'cuda' dacă ai GPU
model.eval()


CommError: artifact 'best_model:model' not found in 'stud_ai_bbu/inferSent-project'